In [ ]:
import argparse

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
import lightgbm as lgb
import matplotlib as mpl

import newron.lightgbm
import newron
remote_server_uri = SERVER_URI # set to your server URI
newron.set_tracking_uri(remote_server_uri)
exp_name = "Lightgbm_Example"
newron.set_experiment(exp_name)

mpl.use("Agg")

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

learning_rate = 0.1
colsample_bytree = 1.0
subsample = 1.0
# enable auto logging
newron.lightgbm.autolog()

train_set = lgb.Dataset(X_train, label=y_train)

with newron.start_run():

    # train model
    params = {
        "objective": "multiclass",
        "num_class": 3,
        "learning_rate": learning_rate,
        "metric": "multi_logloss",
        "colsample_bytree": colsample_bytree,
        "subsample": subsample,
        "seed": 42,
    }
    model = lgb.train(
        params, train_set, num_boost_round=10, valid_sets=[train_set], valid_names=["train"]
    )

    # evaluate model
    y_proba = model.predict(X_test)
    y_pred = y_proba.argmax(axis=1)
    loss = log_loss(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    # log metrics
    newron.log_metrics({"log_loss": loss, "accuracy": acc})
